In [58]:
import logging
import os
from datetime import timedelta
from urllib.request import urlretrieve
from urllib.request import urlopen
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import base64
import webbrowser
from urllib.parse import urlencode
import spotipy
from http.server import HTTPServer,SimpleHTTPRequestHandler

In [63]:
client_id = '<>'
client_secret = '<>'
client_credentials_manager = spotipy.oauth2.SpotifyClientCredentials(client_id, client_secret) 
redirect_uri="http://localhost:7777/callback"
oauth_manager = spotipy.oauth2.SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope='user-read-private')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, oauth_manager=oauth_manager)

In [67]:
data_hackers_shows = sp.search("data hackers",type='show',limit=10,market=data_hackers_shows.get('shows').get('items')[0].get('available_markets'))
show_list = []
columns=['id','name','description','total_episodes']
for item in data_hackers_shows.get('shows').get('items'):
    if item:
        show_list.append({col:item.get(col) for col in columns})
pd.json_normalize(show_list)

,id,name,description,total_episodes
0,1oMIHOXsrLFENAeM743g93,Data Hackers,O podcast oficial do Data Hackers: a maior com...,100
1,1nfBfEtCmuvJkPQgR3bS5a,Data Career Podcast,The Data Career Podcast: helping you break int...,93
2,7r3ijPPBDnBAlN7iv8m8M2,Hidden Files,It aims at creating awareness around hackers s...,100
3,6zkacjxPFG0qbcMWzDHvfH,The Hacker Mind,The Hacker Mind is an original podcast from th...,86
4,7kU6yrZ33zDGmuCLXepEPK,It's All About Data/Internet These Days. Hacke...,Data/Internet security you name it I'm interes...,1
5,21zZfOy7VCSIIWlJ64DElv,Hacked,"Strange tales of hacking, tech, internet grift...",83
6,1BZN7H3ikovSejhwQTzNm4,Data Skeptic,Data Skeptic is your source for a perspective ...,514
7,3YRafqb2OGxfXgoIPfRbe2,Exploit Brokers - Tech and Hacking News Commen...,Exploit brokers is dedicated to tech and hacki...,14
8,5wcjfEn1GxY8fOhjNYs0FV,"The Elite HRV Podcast: Heart Rate Variability,...",Uncommon methods for maximizing health and per...,63


In [41]:
ep_list = []
dh_ep_return = sp.show_episodes(show_id='1oMIHOXsrLFENAeM743g93',offset=0)
columns=['id','name','description','release_date','duration_ms','language','explicit','type']
while dh_ep_return:
    dh_ep = dh_ep_return.get('items')

    for item in dh_ep:
        ep_list.append({col:item.get(col) for col in columns})
    
    if not dh_ep_return['next']:
        break
    dh_ep_return = sp.next(dh_ep_return)
pd.json_normalize(ep_list)

,id,name,description,release_date,duration_ms,language,explicit,type
0,7mvOFL5rvFvIzh3PSXlRRm,Magalu Cloud: Por dentro da primeira Cloud Bra...,"Você já deve ter ouvido, sobre o lançamento da...",2024-01-19,2704143,pt-BR,False,episode
1,2rCM3L0Hc2pL4EvIAXdbsc,Data Hackers News #13 - Microsoft é mais valio...,"Abrindo as notícias do ano 2024, está no ar, o...",2024-01-17,834768,pt-BR,False,episode
2,5BA6A16kyA1UKZg6Do86Jn,Como Big Techs Brasileiras usam Inteligência A...,A percepção de empresas brasileiras que fazem ...,2024-01-12,3697057,pt-BR,False,episode
3,7l8vxJkbHgIWObpFQIqPOw,Data Hackers News #12 - Demonstração do Google...,Está no ar nossa o Data Hackers News ! Os assu...,2023-12-13,909531,pt-BR,False,episode
4,4rLhkrWX6OMt8yv5RL5WpU,Data Hackers News #11 - Lei criada com ChatGPT...,Está no ar nossa o Data Hackers News ! Os assu...,2023-12-06,783804,pt-BR,False,episode
...,...,...,...,...,...,...,...,...
95,7Bg1wXMnQbJMeoJ1RjEXvm,Como criar um time de Data Science - Episódio ...,"Fala, Data Hackers. Sejam muito bem-vindos a m...",2019-01-06,3522900,pt-BR,False,episode
96,7jV5hVnSDQtRGpO3kPdLxV,O que um Engenheiro de Dados faz? - Episódio 0...,"Olá, Data Hackers. Sejam muito bem-vindos a ma...",2018-12-09,3641940,pt-BR,False,episode
97,11mVafwx00vWHOLf5kiYLY,Data-Driven Products: por dentro da área de pr...,Sejam muito bem-vindos a mais um episódio do s...,2018-11-19,2963328,pt-BR,False,episode
98,7kfQHi3sxNH3RhZXZL8i9X,Como é o dia a dia de um Cientista de Dados - ...,"Olá, Data Hacker! Bem-vindo a mais um episódio...",2018-10-14,3589826,pt-BR,False,episode


In [68]:
auth_headers = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "user-library-read"
}

# r = requests.get('https://accounts.spotify.com/authorize', params=auth_headers)
# webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))

r = urlopen("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))

In [69]:
code = '<>'

payload = {
    "q":"data%20hackers",
    "type":"show",
    "limit":50
}

encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

header = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}

r = requests.post('https://accounts.spotify.com/api/token', headers=header, data=data)
r.text

'{"error":"invalid_grant","error_description":"Invalid authorization code"}'

In [70]:
token = '<>'

header = {
    "Authorization": "Bearer " + token,
}

data = {
    "q": "data hacker",
    "type": "show",
}

r = requests.get('https://api.spotify.com/v1/search?query=data%20hacker&type=show&offset=0&limit=10', headers=header) #, params=data)
r.json()

{'error': {'status': 400,
  'message': 'Only valid bearer authentication supported'}}

In [6]:
for file in sales_files:

    year = file.get('year')
    url = file.get('url').replace(" ", "%20")
    file_name =  os.path.join(sales_folder, file.get('file_name'))

    paramns = {'folder_name': sales_folder,
               'file_name': file_name,
               'url': url

               }
    donwload_sales_file(**paramns)

In [7]:
df = pd.read_excel(file_name)  

In [8]:
df['year'] = 2019

In [19]:
df.columns = [c.lower() for c in  df.columns]

In [20]:
df

,id_marca,marca,id_linha,linha,data_venda,qtd_venda,year
0,4,VULT,2,PERFUMARIA,2019-01-15,7,2019
1,4,VULT,2,PERFUMARIA,2019-10-12,2,2019
2,3,QUEM DISSE BERENICE,5,HIDRATANTES,2019-09-01,3,2019
3,3,QUEM DISSE BERENICE,3,MAQUIAGEM,2019-07-29,17,2019
4,1,BOTICÁRIO,4,SOLAR,2019-11-15,19,2019
...,...,...,...,...,...,...,...
995,1,BOTICÁRIO,2,PERFUMARIA,2019-11-19,14,2019
996,5,BELEZA NA WEB,2,PERFUMARIA,2019-12-14,5,2019
997,2,EUDORA,4,SOLAR,2019-10-03,9,2019
998,3,QUEM DISSE BERENICE,3,MAQUIAGEM,2019-03-03,20,2019
